# Assigment 7 | Applied Machine Learning | Paras Ahuja

### Continuing to the previous machine learning problem, let's get back to the pre-processed dataset Suicide Rates Overview 1985 to 2016 file. We would like to have a machine learning model to predict the suicide rate 'suicides/100k pop'.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer as ms, mean_absolute_error as mae, mean_squared_error as mse
from sklearn.model_selection import cross_validate, StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

In [2]:
df = pd.read_csv('master.csv')
df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [3]:
df.rename(columns={" gdp_for_year ($) ": "gdp_for_year", "gdp_per_capita ($)": "gdp_per_capita"}, inplace=True)
df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year,gdp_per_capita,generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [4]:
df = df.drop(columns=["country", "suicides_no", "population", "country-year", "HDI for year", "gdp_for_year"])
df.head()

,year,sex,age,suicides/100k pop,gdp_per_capita,generation
0,1987,male,15-24 years,6.71,796,Generation X
1,1987,male,35-54 years,5.19,796,Silent
2,1987,female,15-24 years,4.83,796,Generation X
3,1987,male,75+ years,4.59,796,G.I. Generation
4,1987,male,25-34 years,3.28,796,Boomers


In [5]:
df['sex'] = df['sex'].replace({'female':0, 'male':1})
df.head()

,year,sex,age,suicides/100k pop,gdp_per_capita,generation
0,1987,1,15-24 years,6.71,796,Generation X
1,1987,1,35-54 years,5.19,796,Silent
2,1987,0,15-24 years,4.83,796,Generation X
3,1987,1,75+ years,4.59,796,G.I. Generation
4,1987,1,25-34 years,3.28,796,Boomers


In [6]:
switch = df['suicides/100k pop']
df.drop(labels=['suicides/100k pop'], axis=1, inplace = True)
df.insert(3, 'suicides/100k pop', switch)
df.head()

,year,sex,age,suicides/100k pop,gdp_per_capita,generation
0,1987,1,15-24 years,6.71,796,Generation X
1,1987,1,35-54 years,5.19,796,Silent
2,1987,0,15-24 years,4.83,796,Generation X
3,1987,1,75+ years,4.59,796,G.I. Generation
4,1987,1,25-34 years,3.28,796,Boomers


In [7]:
target = pd.DataFrame(data=[['15-24 years', 'male', 'Generation X']], columns=['age', 'sex', 'generation'])
ohe = pd.get_dummies(df, columns=["age", "generation"])
ohe.head()

,year,sex,suicides/100k pop,gdp_per_capita,age_15-24 years,age_25-34 years,age_35-54 years,age_5-14 years,age_55-74 years,age_75+ years,generation_Boomers,generation_G.I. Generation,generation_Generation X,generation_Generation Z,generation_Millenials,generation_Silent
0,1987,1,6.71,796,1,0,0,0,0,0,0,0,1,0,0,0
1,1987,1,5.19,796,0,0,1,0,0,0,0,0,0,0,0,1
2,1987,0,4.83,796,1,0,0,0,0,0,0,0,1,0,0,0
3,1987,1,4.59,796,0,0,0,0,0,1,0,1,0,0,0,0
4,1987,1,3.28,796,0,1,0,0,0,0,1,0,0,0,0,0


In [8]:
targets = ohe.loc[(ohe['age_15-24 years'] == 1) & (ohe["sex"] == 1) & (ohe["generation_Generation X"] == 1)]
targets.head()

,year,sex,suicides/100k pop,gdp_per_capita,age_15-24 years,age_25-34 years,age_35-54 years,age_5-14 years,age_55-74 years,age_75+ years,generation_Boomers,generation_G.I. Generation,generation_Generation X,generation_Generation Z,generation_Millenials,generation_Silent
0,1987,1,6.71,796,1,0,0,0,0,0,0,0,1,0,0,0
13,1988,1,5.33,769,1,0,0,0,0,0,0,0,1,0,0,0
28,1989,1,3.71,833,1,0,0,0,0,0,0,0,1,0,0,0
37,1992,1,3.41,251,1,0,0,0,0,0,0,0,1,0,0,0
48,1993,1,7.40,437,1,0,0,0,0,0,0,0,1,0,0,0


In [9]:
targets.shape

(1057, 16)

### 1. Keep the variables as one-hot encoded and develop a multiple linear regression model. Use your model to predict the target variable for the people with age 20, male, and generation X. What is the MAE error of this prediction? How many line coefficients are there?

In [10]:
def mae(y, y_pred):
    return (len(y)**-1) * np.sum(np.abs(y_pred - y))

In [11]:
X = ohe.loc[:, ohe.columns != "suicides/100k pop"].values
y = ohe.loc[:, ohe.columns == "suicides/100k pop"].values.ravel()
print(X.shape, y.shape)

(27820, 15) (27820,)


In [12]:
lr = LinearRegression()
lr.fit(X, y)
y_pred = lr.predict(X)
print("Model MAE: {0:.3f}".format(mae(y, y_pred)))
print("Model Regression Score: {0:.3f}".format(lr.score(X, y)))
print("Number of Hyperplane Coefficients: {0:.3f}".format(len(lr.coef_)))

Model MAE: 10.155
Model Regression Score: 0.296
Number of Hyperplane Coefficients: 15.000


In [13]:
year_mean, gdp_per_capita_mean = np.mean(X[:,0]), np.mean(X[:,3])
X_test = np.array([year_mean, 1, gdp_per_capita_mean, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], ndmin=2)
with np.printoptions(precision=3, suppress=True):
    print(X_test.shape, X_test)

(1, 15) [[2001.258    1.       0.167    1.       0.       0.       0.       0.
     0.       0.       0.       1.       0.       0.       0.   ]]


In [14]:
y_test_pred = lr.predict(X_test)
print("Prediction for Generation X 20 year old male: {0:.3f}".format(y_test_pred[0]))

Prediction for Generation X 20 year old male: 15.875


In [15]:
y_test_mean = targets.iloc[:,2].mean()
print("Given male, age 20, gen X, y average: {0:.3f}".format(y_test_mean))

Given male, age 20, gen X, y average: 14.513


In [16]:
print("Prediction MAE: {0:.3f}".format(mae(np.array(y_test_mean, ndmin=2), y_test_pred)))

Prediction MAE: 1.362


Predition MAE is 1.362 and model MAE is 10.155. We have 15 coefficients.

### 2. Now use the original sex, age and generation variables in numerical form and develop a new model. Use your model to predict the target value for the people with age 20, male, and generation X. What is the MAE error of this prediction? How many line coefficients are there? (Note that for this step you have to think of a way of encoding the original nominal age feature and generation feature into numerical features.)

In [17]:
df.head()

,year,sex,age,suicides/100k pop,gdp_per_capita,generation
0,1987,1,15-24 years,6.71,796,Generation X
1,1987,1,35-54 years,5.19,796,Silent
2,1987,0,15-24 years,4.83,796,Generation X
3,1987,1,75+ years,4.59,796,G.I. Generation
4,1987,1,25-34 years,3.28,796,Boomers


In [18]:
generation = {"G.I. Generation": 1, "Silent": 2, "Boomers": 3, "Generation X": 4, "Millenials": 5, "Generation Z": 6}
df["age"] = df["age"].replace({'5-14 years':10, '15-24 years':20, '25-34 years':30, '35-54 years':45, '55-74 years':65, '75+ years':80})
df["generation"] = df["generation"].replace(generation)
df.head()

,year,sex,age,suicides/100k pop,gdp_per_capita,generation
0,1987,1,20,6.71,796,4
1,1987,1,45,5.19,796,2
2,1987,0,20,4.83,796,4
3,1987,1,80,4.59,796,1
4,1987,1,30,3.28,796,3


In [19]:
X = df.loc[:, df.columns != "suicides/100k pop"].values
y = df.loc[:, df.columns == "suicides/100k pop"].values.ravel()
print(X.shape, y.shape)

(27820, 5) (27820,)


In [20]:
lr = LinearRegression()
lr.fit(X, y)
y_pred = lr.predict(X)
print("Model MAE: {0:.3f}".format(mae(y, y_pred)))
print("Model Regression Score: {0:.3f}".format(lr.score(X, y)))
print("Number of Hyperplane Coefficients: {0:.3f}".format(len(lr.coef_)))

Model MAE: 10.321
Model Regression Score: 0.280
Number of Hyperplane Coefficients: 5.000


In [21]:
year_mean, gdp_per_capita_mean = np.mean(X[:,0]), np.mean(X[:,4])
X_test = np.array([year_mean, 1, 20, gdp_per_capita_mean, 4], ndmin=2)
with np.printoptions(precision=3, suppress=True):
    print(X_test.shape, X_test)

(1, 5) [[2001.258    1.      20.       3.383    4.   ]]


In [22]:
y_test_pred = lr.predict(X_test)
print("Prediction for Generation X 20 year old male: {0:.3f}".format(y_test_pred[0]))

Prediction for Generation X 20 year old male: 14.075


In [23]:
targets = df.loc[(df["age"] == 20) & (df["generation"] == 4) & (df["sex"] == 1)]
targets.shape

(1057, 6)

In [24]:
y_test_mean = targets.iloc[:,3].mean()
print("Given male, age 20, gen X, y average: {0:.3f}".format(y_test_mean))

Given male, age 20, gen X, y average: 14.513


In [25]:
print("Prediction MAE: {0:.3f}".format(mae(np.array(y_test_mean, ndmin=2), y_test_pred)))

Prediction MAE: 0.438


Predition MAE is 0.438 and model MAE is 10.321. We have 5 coefficients.

### 3. Did you note any change in these two model performances?

There are a few things to note here. Firstly there are less number of coefficients when it comes to hyperplane dimensions, which seems to help. Note also that less number of coefficients will have less errors. We can witness this fact by comparing the MAE's. As we notice in our predictions above, the error between the average y and the predicted y is smaller. Lastly, conversion of nominal variables to numerical must be precise or we will be introducing errors.

### 4. What is the prediction for age 33, male and generation Alpha (i.e. the generation after generation Z)?

Note that the generations we have only go up til number 6 (nominally speaking). In order to accomodate for Generation Alpha, we will use 7. 

In [26]:
X_test = np.array([year_mean, 1, 33, gdp_per_capita_mean, 7], ndmin=2)
y_test_pred = lr.predict(X_test)
print("Prediction for Generation Alpha 33 year old male: {0:.3f}".format(y_test_pred[0]))

Prediction for Generation Alpha 33 year old male: 17.395


So we have that our prediction is 17.395 for age 33 male of generation Alpha.

### 5. Give one advantage when using regression (as opposed to classification with nominal features) in terms of input data features.

Note that when it comes to regression vs classification; classification is the task of predicting a discrete class label; regression is the task of predicting a continuous quantity.
With regression it is much easier to witness the effect of an input feature on the target value since the relation is being modeled by a hypterplane. Furthermore, for input data feature, new levels that are half-way between two levels can be easily encoded.

### 6. Give one advantage when using regular numerical values rather than one-hot encoding for regression.

For numerical features never seen before feature values can be used. This provides a bit more clarity/transperancy when it comes to generalization of an algorithm. For nominal never seen before levels can be encoded, assuming there is a relation between the levels of features.

### 7. Now that you developed both a classifier and a regression model for this problem, which method do you suggest to your machine learning model customer and why?

We should recommend regression on this dataset with numerical target variable than classification. This is because sometimes the simlest relation such as a hyperplan fit is the best model. If we have a really large dataset then we can use neural networks but for this problem regression should be recommended. Furthermore, also consider the fact that classification is better choice when the desired output is a discrete label; helpful when the answer to our question falls under a finite set of possible outcomes. In order to use classification on this dataset, we would have to convert our prediction feature - suicides/100k pop - to accomodate for classification. No matter how hard we try we will lose some information in this conversion. Therefore, this is another reason to use regression and preseve the "original" form.